In [1]:
import os
os.environ['KIVY_NO_ARGS'] = '1'

In [2]:
from kivy.app import App
from kivy.uix.progressbar import ProgressBar
from kivy.lang import Builder
from kivy.uix.screenmanager import ScreenManager, Screen
from kivy.animation import Animation
from kivy.core.window import Window
from kivy_garden.mapview import MapView, MapMarkerPopup,MapLayer,MapMarker
from google.cloud import firestore
import threading
from google.oauth2 import service_account
from PIL import Image
import time
from kivy.uix.popup import Popup
from kivy.properties import ObjectProperty
from kivy_garden.mapview.geojson import GeoJsonMapLayer
from kivy.uix.label import Label
from kivy.properties import NumericProperty
from kivy.uix.widget import Widget
from kivy.properties import NumericProperty, ListProperty
from kivy.uix.behaviors import ButtonBehavior
import math

[INFO   ] [Logger      ] Record log in C:\Users\BUSE\.kivy\logs\kivy_24-05-08_59.txt
[INFO   ] [deps        ] Successfully imported "kivy_deps.angle" 0.4.0
[INFO   ] [deps        ] Successfully imported "kivy_deps.glew" 0.3.1
[INFO   ] [deps        ] Successfully imported "kivy_deps.sdl2" 0.7.0
[INFO   ] [Kivy        ] v2.3.0
[INFO   ] [Kivy        ] Installed at "c:\Users\BUSE\Anaconda3\lib\site-packages\kivy\__init__.py"
[INFO   ] [Python      ] v3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
[INFO   ] [Python      ] Interpreter at "c:\Users\BUSE\Anaconda3\python.exe"
[INFO   ] [Logger      ] Purge log fired. Processing...
[INFO   ] [Logger      ] Purge finished!
[INFO   ] [Factory     ] 195 symbols loaded
[INFO   ] [Image       ] Providers: img_tex, img_dds, img_sdl2, img_pil (img_ffpyplayer ignored)
[INFO   ] [Window      ] Provider: sdl2
[INFO   ] [GL          ] Using the "OpenGL" graphics system
[INFO   ] [GL          ] GLEW initialization succeeded
[INFO   ] 

In [3]:
global geo_state,glo_heart_rate
geo_state='normal'
glo_heart_rate=120

# Specify the path to your service account key file
service_account_path = "C:\\Users\\Buse\\Anaconda3\\Lib\\site-packages\\zollyapp-6ef028f890fb.json"

credentials = service_account.Credentials.from_service_account_file(service_account_path)
db = firestore.Client(credentials=credentials, project=credentials.project_id)


def fetch_location_data(update_ui_callback):
    doc_ref = db.collection('your_collection').document('your_document')  # Adjust as needed

    try:
        doc = doc_ref.get()
        if doc.exists:
            data = doc.to_dict()
            lat = data.get('lat', 0)
            lng = data.get('lng', 0)
            steps = data.get('steps', 0)  # Fetch steps
            heart = data.get('heart', 0)
            # Update the UI with the fetched data
            
            if lat=='N/A' or lng== 'N/A':
                lat=39.872121
                lng=32.751761
            update_ui_callback(lat, lng, steps,heart)

        else:
            print('No such document!')
    except Exception as e:
        print(f'An error occurred: {e}')
        
def update_cloud_data(field, value):
    try:
        doc_ref = db.collection('your_collection').document('your_document')
        doc = doc_ref.get()
        if doc.exists:
            data = doc.to_dict()
            lat = data.get('lat', 0)
            lng = data.get('lng', 0)
            steps = data.get('steps', 0)  # Fetch steps
            heart = data.get('heart', 0)
        doc_ref.set({
            field: value,
            'lat': lat,
            'lng' : lng,
            'steps' : steps,
            'heart' :heart
        })
        print("Document successfully updated!")
    except Exception as e:
        print(f'An error occurred: {e}')

 
def resize_marker_image(source_path, output_path, size=(50, 50)):
    image = Image.open(source_path)
    
    # Dynamically determine the resampling filter based on what's available
    # This approach avoids direct reference to Image.ANTIALIAS or ImageResampling
    try:
        # Pillow >= 8.0.0
        resample_filter = Image.Resampling.LANCZOS
    except AttributeError:
        try:
            # Pillow < 8.0.0 (where Image.ANTIALIAS is available)
            resample_filter = Image.LANCZOS
        except AttributeError:
            # Fallback for older Pillow versions where LANCZOS might not be available
            resample_filter = Image.NEAREST
    
    image.thumbnail(size, resample_filter)
    image.save(output_path)

resize_marker_image("C:\\Users\\Buse\\Anaconda3\\Lib\\site-packages\\marker.png","C:\\Users\\Buse\\Anaconda3\\Lib\\site-packages\\resized_marker.png")

# Make sure you're referencing the resized marker image
marker_source = "C:\\Users\\Buse\\Anaconda3\\Lib\\site-packages\\resized_marker.png"

Window.size = (350, 500)
Window.clearcolor = (1,1,1)  # Set the window background color (light gray in this example)

# Define the Kivy UI
kv = """
ScreenManager:
    LoginScreen:
    MainMenuScreen:
    LocationScreen:
    HealthScreen:
    ActivityTrackingScreen:
    SettingsScreen:
    GoalsScreen:
    AlertsScreen:
    PetInfoScreen:   
    WiFiSettingsScreen:
        
<LoginScreen>:
    name: 'login'
    Label:
        text: 'PetTracker App'
        font_size: 48
        pos_hint: {'center_x': 0.5, 'center_y': 0.55}
        color: 0.188, 0.616, 0
        
    BoxLayout:
        orientation: 'vertical'
        size_hint: None, None
        size: 300, 200
        pos_hint: {'center_x': 0.5, 'center_y': 0.35}


        TextInput:
            id: username_input
            hint_text: 'Username'
            size_hint_y: None
            height: 40
        TextInput:
            id: password_input
            hint_text: 'Password'
            password: True
            size_hint_y: None
            height: 40
        Button:
            text: 'Login'
            on_release: root.login(username_input.text, password_input.text)
            background_normal: ''
            background_down: ''
            background_color: 0,0,0
            color: 1,1,1
            size_hint_y: None
            height: 40
        Label:
            id: login_status
            text: ''
            color: 0.769, 0.106, 0.106

<MainMenuScreen>:
    name: 'main_menu'
    BoxLayout:
        orientation: 'vertical'
        BoxLayout:
            orientation: 'horizontal'
            size_hint_y: None
            height: 50

            Button:
                text: 'Settings'
                on_release: app.root.current = 'Settings'
                size_hint_x: None
                width: 100
                background_normal: ''
                background_down: ''
                background_color: 0,0,0
                color: 1,1,1
            
            Button:
                text: 'Pet Tracker App'
                on_release: app.root.current = 'main_menu'
                size_hint_x: 1
                halign: 'center'
                background_normal: ''
                background_down: ''
                background_color: 0,0,0
                color: 1,1,1
            
            Button:
                text: 'Logout'
                on_release: app.root.current = 'login'
                size_hint_x: None
                width: 100
                background_normal: ''
                background_down: ''
                background_color: 0,0,0
                color: 1,1,1
        Button:
            text: 'Location'
            font_size: '18sp'
            on_release: app.root.current = 'location'
            background_normal: ''
            background_down: ''
            color: 1,1,1
            background_color: 0.188, 0.616, 0, 0.9
        Button:
            text: 'Health'
            font_size: '18sp'
            on_release: app.root.current = 'health'
            background_normal: ''
            background_down: ''
            color: 1,1,1
            background_color: 0.188, 0.616, 0, 0.75
        Button:
            text: 'Activity Tracking'
            font_size: '18sp'
            on_release: app.root.current = 'activity_tracking'
            background_normal: ''
            background_down: ''
            color: 1,1,1
            background_color: 0.188, 0.616, 0, 0.6
            
            
<AlertDialog>:
    size_hint: None, None
    size: 400, 200
    title: 'Alert'
    BoxLayout:
        orientation: 'vertical'
        Label:
            text: root.message
        Button:
            text: 'Close'
            on_release: root.dismiss_popup()

<LocationScreen>:
    name: 'location'
    BoxLayout:
        orientation: 'vertical'
        BoxLayout:
            orientation: 'horizontal'
            size_hint_y: None
            height: 50

            Button:
                text: 'Settings'
                on_release: app.root.current = 'Settings'
                size_hint_x: None
                width: 100
                background_normal: ''
                background_down: ''
                background_color: 0,0,0
                color: 1,1,1
            
            Button:
                text: 'Back to Main Menu'
                on_release: app.root.current = 'main_menu'
                size_hint_x: 1
                halign: 'center'
                background_normal: ''
                background_down: ''
                background_color: 0,0,0
                color: 1,1,1
            
            Button:
                text: 'Logout'
                on_release: app.root.current = 'login'
                size_hint_x: None
                width: 100
                background_normal: ''
                background_down: ''
                background_color: 0,0,0
                color: 1,1,1
        
        MapView:
            id:mapview
            lat: 39.87  # Set the latitude of the initial map center
            lon: 32.85  # Set the longitude of the initial map center
            zoom: 13  # Set the initial zoom level
            size_hint: 1, 0.9  # Adjust the size of the map
        BoxLayout:
            orientation: 'vertical'
            size_hint_y: None
            height: 100
            TextInput:
                id: radius_input
                hint_text: 'Enter radius in meters'
                input_filter: 'int'
                size_hint_y: None
                height: 50
            Button:
                text: 'Set Safe Zone'
                size_hint_y: None
                height: 50
                background_normal: ''
                background_down: ''
                background_color: 0.188, 0.616, 0
                color: 1,1,1
                on_release: root.update_safe_zone_radius(radius_input.text)
                 
                
<AlertPopup>:
    size_hint: None, None
    size: 300, 200
    title: "Alert"
    auto_dismiss: False
    BoxLayout:
        orientation: 'vertical'
        Label:
            id: alert_text
            text: ""
            size_hint_y: 0.8
        Button:
            text: 'OK'
            on_release: root.dismiss()
            size_hint_y: 0.2

<HeartBeatWidget@Widget>:
    canvas.before:
        Color:
            rgba: 0.769, 0.106, 0.106, 1  
        Ellipse:
            pos: self.center_x - self.size_x / 3.9, self.center_y - self.size_y / 5.3
            size: self.size_x/1.8, self.size_y/1.8
        Ellipse:
            pos: self.center_x + self.size_x /5, self.center_y - self.size_y / 5.3
            size: self.size_x/1.8, self.size_y/1.8
        Triangle:
            points: [self.center_x - self.size_x / 4, self.center_y, self.center_x + self.size_x / 4 * 3, self.center_y, self.center_x + self.size_x / 4, self.center_y - self.size_y / 2]
    size_x: 100
    size_y: 100

<MapCiecleWidget@Widget>:
    canvas:
        Color:
            rgba: 0.769, 0.106, 0.106, 0.4  
        Ellipse:
            pos: self.center_x , self.center_y
            size: self.size_x, self.size_y
    size_x: 100
    size_y: 100

<HealthScreen>:
    name: 'health'
    BoxLayout:
        orientation: 'vertical'                        
        BoxLayout:
            orientation: 'horizontal'
            size_hint_y: None
            height: 50

            Button:
                text: 'Settings'
                on_release: app.root.current = 'Settings'
                size_hint_x: None
                width: 100
                background_normal: ''
                background_down: ''
                background_color: 0,0,0
                color: 1,1,1
            
            Button:
                text: 'Back to Main Menu'
                on_release: app.root.current = 'main_menu'
                size_hint_x: 1
                halign: 'center'
                background_normal: ''
                background_down: ''
                background_color: 0,0,0
                color: 1,1,1
            
            Button:
                text: 'Logout'
                on_release: app.root.current = 'login'
                size_hint_x: None
                width: 100
                background_normal: ''
                background_down: ''
                background_color: 0,0,0
                color: 1,1,1
        FloatLayout:
            orientation: 'horizontal'
            size_hint_y: None
            height: 250
            HeartBeatWidget:
                id: heart
                size_hint: None, None
                size: 100, 100
                pos_hint: {'center_x': 0.44, 'center_y': 0.5}
        BoxLayout:
            Label:
                id: heart_rate
                text: 'Heart rate: 0'
                font_size: '20sp'
                size_hint_y: None
                height: 100
                color: 0.188, 0.616, 0
        BoxLayout:
            orientation: 'horizontal'
            size_hint_y: None
            height: 350
                
<CustomProgressBar>:
    canvas:
        Color:
            rgba: 0.5, 0.5, 0.5, 1  # Gray color for the background
        Rectangle:
            size: self.width, dp(20)  # Thickness of the progress bar
            pos: self.x, self.center_y - dp(5)  # Position the progress bar in the middle of the height
        Color:
            rgba: 0.188, 0.616, 0, 1  # Green color for the progress
        Rectangle:
            size: self.width * self.value/100, dp(20)  # Adjust width based on the progress
            pos: self.pos[0], self.center_y - dp(5)  
            
<ActivityTrackingScreen>:
    on_pre_enter: root.compare_goal_with_steps()
    name: 'activity_tracking'
    BoxLayout:
        orientation: 'vertical'

        BoxLayout:
            orientation: 'horizontal'
            size_hint_y: None
            height: 50

            Button:
                text: 'Settings'
                on_release: app.root.current = 'Settings'
                size_hint_x: None
                width: 100
                background_normal: ''
                background_down: ''
                background_color: 0,0,0
                color: 1,1,1
            
            Button:
                text: 'Back to Main Menu'
                on_release: app.root.current = 'main_menu'
                size_hint_x: 1
                halign: 'center'
                background_normal: ''
                background_down: ''
                background_color: 0,0,0
                color: 1,1,1
            
            Button:
                text: 'Logout'
                on_release: app.root.current = 'login'
                size_hint_x: None
                width: 100
                background_normal: ''
                background_down: ''
                background_color: 0,0,0
                color: 1,1,1
                
        Label:
            id: step_count
            text: 'Steps: 0'
            font_size: '20sp'
            color: 0.188, 0.616, 0
        CustomProgressBar:
            id: custom_progress_bar
            max: 100
            value: 10  # Initial value, will be updated dynamically
            size_hint_y: 0.03
            size_hint_x: 0.8  # Set the width of the progress bar
            height: dp(10) 
            pos_hint: {'center_x': 0.5}  
        Label:
            id: distance_tracked
            text: 'Distance: 0 meters'
            font_size: '20sp'
            color: 0.188, 0.616, 0
            

 
<SettingsScreen>:
    name: 'Settings'
    BoxLayout:
        orientation: 'vertical'
        BoxLayout:
            orientation: 'horizontal'
            size_hint_y: None
            height: 50

            Button:
                text: 'Settings'
                on_release: app.root.current = 'Settings'
                size_hint_x: None
                width: 100
                background_normal: ''
                background_down: ''
                background_color: 0,0,0
                color: 1,1,1
            
            Button:
                text: 'Back to Main Menu'
                on_release: app.root.current = 'main_menu'
                size_hint_x: 1
                halign: 'center'
                background_normal: ''
                background_down: ''
                background_color: 0,0,0
                color: 1,1,1
            
            Button:
                text: 'Logout'
                on_release: app.root.current = 'login'
                size_hint_x: None
                width: 100
                background_normal: ''
                background_down: ''
                background_color: 0,0,0
                color: 1,1,1
        BoxLayout:
            orientation: 'vertical'
            Button:
                text: 'Pet Information'
                on_release: app.root.current = 'pet_info'
                font_size: '18sp'
                background_normal: ''
                background_down: ''
                background_color: 0.188, 0.616, 0, 0.9
                color: 0.965, 0.949, 0.941
                size_hint_y: 0.25 
            Button:
                text: 'Alerts / Notifications'
                font_size: '18sp'
                on_release: app.root.current = 'alerts'
                background_normal: ''
                background_down: ''
                color: 1, 1, 1
                background_color: 0.188, 0.616, 0, 0.8
                size_hint_y: 0.25 
            Button:
                text: 'Daily Goals'
                font_size: '18sp'
                on_release: app.root.current = 'daily_goals'
                background_normal: ''
                background_down: ''
                color: 1, 1, 1
                background_color: 0.188, 0.616, 0, 0.7
                size_hint_y: 0.25  
            Button:
                text: 'WiFi Settings'
                font_size: '18sp'
                background_normal: ''
                background_down: ''
                background_color: 0.188, 0.616, 0, 0.6
                color: 1, 1, 1
                size_hint_y: 0.25  
                on_press: app.root.current = 'wifi_settings'

                    
                        

        
            
<GoalsScreen>:
    name: 'daily_goals'
    BoxLayout:
        orientation: 'vertical'
        BoxLayout:
            orientation: 'horizontal'
            size_hint_y: None
            height: 50

            Button:
                text: 'Settings'
                on_release: app.root.current = 'Settings'
                size_hint_x: None
                width: 100
                background_normal: ''
                background_down: ''
                background_color: 0,0,0
                color: 1,1,1
            
            Button:
                text: 'Back to Main Menu'
                on_release: app.root.current = 'main_menu'
                size_hint_x: 1
                halign: 'center'
                background_normal: ''
                background_down: ''
                background_color: 0,0,0
                color: 1,1,1
            
            Button:
                text: 'Logout'
                on_release: app.root.current = 'login'
                size_hint_x: None
                width: 100
                background_normal: ''
                background_down: ''
                background_color: 0,0,0
                color: 1,1,1
                
                
        BoxLayout:
            orientation: 'vertical'
            size_hint_y: None
            height: 100
            TextInput:
                id: goal_input
                hint_text: 'Enter daily activity goal in number of steps'
                input_filter: 'int'
                size_hint_y: None
                height: 50
            Button:
                text: 'Set Daily Goal'
                size_hint_y: None
                height: 50
                background_normal: ''
                background_down: ''
                background_color: 0.188, 0.616, 0
                color: 1,1,1
                on_release: root.update_daily_goal(goal_input.text)
        BoxLayout:
            orientation: 'vertical'

<AlertsScreen>:
    name: 'alerts'
    BoxLayout:
        orientation: 'vertical'
        BoxLayout:
            orientation: 'horizontal'
            size_hint_y: None
            height: 50

            Button:
                text: 'Settings'
                on_release: app.root.current = 'Settings'
                size_hint_x: None
                width: 100
                background_normal: ''
                background_down: ''
                background_color: 0,0,0
                color: 1,1,1
            
            Button:
                text: 'Back to Main Menu'
                on_release: app.root.current = 'main_menu'
                size_hint_x: 1
                halign: 'center'
                background_normal: ''
                background_down: ''
                background_color: 0,0,0
                color: 1,1,1
            
            Button:
                text: 'Logout'
                on_release: app.root.current = 'login'
                size_hint_x: None
                width: 100
                background_normal: ''
                background_down: ''
                background_color: 0,0,0
                color: 1,1,1
        BoxLayout:
            orientation: 'vertical'
            ToggleButton:
                id:geo alert
                text: 'Geo Alert'
                size_hint: 1, None
                width: 520
                height: 150
                background_normal: ''
                background_down: ''
                color: 1,1,1
                background_color: 0.188, 0.616, 0 
                on_state: root.on_toggle_button_state_geo(self.state)
            Label:
                id: geo_alert_status
                text: ''
                color: 0.769, 0.106, 0.106
            ToggleButton:
                id: health alert
                text: 'Health Alert'
                size_hint: 1, None
                width: 520
                height: 150
                background_normal: ''
                background_down: ''
                color: 1,1,1
                background_color: 0.188, 0.616, 0
                on_state: root.on_toggle_button_state_health(self.state)
            Label:
                id: health_alert_status
                text: ''
                color: 0.769, 0.106, 0.106
            ToggleButton:
                id: activity alert
                text: 'Extreme Activity Alert'
                size_hint: 1, None
                width: 520
                height: 150
                background_normal: ''
                background_down: ''
                color: 1,1,1
                background_color: 0.188, 0.616, 0
                on_state: root.on_toggle_button_state_activity(self.state)
            Label:
                id: activity_alert_status
                text: ''
                color: 0.769, 0.106, 0.106

<PetInfoScreen>:
    name: 'pet_info'
    BoxLayout:
        orientation: 'vertical'

        # Top Navigation Bar
        BoxLayout:
            size_hint_y: None
            height: 50
            canvas.before:
                Color:
                    rgba: 0, 0, 0, 1
                Rectangle:
                    size: self.size
                    pos: self.pos

            Button:
                text: 'Settings'
                on_release: app.root.current = 'Settings'
                size_hint_x: None
                width: 100
                background_color: 0, 0, 0, 1
                color: 1, 1, 1

            Button:
                text: 'Back to Main Menu'
                on_release: app.root.current = 'main_menu'
                size_hint_x: 1
                background_color: 0, 0, 0, 1
                color: 1, 1, 1

            Button:
                text: 'Logout'
                on_release: app.root.current = 'login'
                size_hint_x: None
                width: 100
                background_color: 0, 0, 0, 1
                color: 1, 1, 1

        # Content Area
        BoxLayout:
            orientation: 'vertical'
            
            padding: [30, 50, 30, 50]
            spacing: 10

           # Label:
            #    text: 'Enter Pet Information'
             #   size_hint_y: None
              #  height: 600
               # font_size: '20sp'
                #color: 0.188, 0.616, 0

            Spinner:
                id: pet_type
                text: 'Select Pet Type'
                values: ['Dog', 'Cat']
                size_hint_y: None
                background_color: 1,1,1
                color: 1,1,1
                height: 60
                on_text: root.on_pet_type_select(self.text) 


            Spinner:
                id: pet_breed
                text: 'Select Breed'
                values: []
                size_hint_y: None
                color: 1,1,1
                height: 60
                background_color: 1,1,1

            TextInput:
                id: pet_age
                hint_text: 'Age'
                input_filter: 'int'
                color: 1,1,1
                size_hint_y: None
                height: 60
                background_color:1,1,1

            Spinner:
                id: pet_gender
                text: 'Select Gender'
                background_color: 1,1,1
                color: 1,1,1
                values: ['Male', 'Female']
                size_hint_y: None
                height: 60

            Button:
                text: 'Save'
                size_hint_y: None
                height: 70
                background_color: 0, 1, 0, 0.9
                color: 1,1,1
                on_release: root.save_pet_info(pet_type.text, pet_breed.text, pet_age.text, pet_gender.text)

<WiFiSettingsScreen>:
    name: 'wifi_settings'
    BoxLayout:
        orientation: 'vertical'

        # Top Navigation Bar
        BoxLayout:
            size_hint_y: None
            height: 50
            canvas.before:
                Color:
                    rgba: 0, 0, 0, 1
                Rectangle:
                    size: self.size
                    pos: self.pos

            Button:
                text: 'Settings'
                on_release: app.root.current = 'Settings'
                size_hint_x: None
                width: 100
                background_color: 0, 0, 0, 1
                color: 1, 1, 1

            Button:
                text: 'Back to Main Menu'
                on_release: app.root.current = 'main_menu'
                size_hint_x: 1
                background_color: 0, 0, 0, 1
                color: 1, 1, 1

            Button:
                text: 'Logout'
                on_release: app.root.current = 'login'
                size_hint_x: None
                width: 100
                background_color: 0, 0, 0, 1
                color: 1, 1, 1

        BoxLayout:
            orientation: 'vertical'
            spacing: 10
            padding: [10, 50, 10, 50]

            Label:
                text: 'WiFi Settings'
                font_size: 20
                size_hint_y: None
                height: 40

            TextInput:
                id: wifi_ssid
                hint_text: 'Enter WiFi SSID'
                size_hint_y:None
                height: 40
                multiline: False

            TextInput:
                id: wifi_password
                hint_text: 'Enter WiFi Password'
                password: True
                size_hint_y: None
                height: 40
                multiline: False
                color: 1,1,1

            Button:
                text: 'Save WiFi Settings'
                size_hint_y: None
                height: 50
                background_color: 0, 1, 0, 0.9
                color: 1,1,1
                on_release:
                    root.save_wifi_settings(wifi_ssid.text, wifi_password.text)
                    app.root.current = 'Settings'  

"""
class AlertDialog(Popup):
    message = "Location is outside the safe zone"

    def __init__(self, message, **kwargs):
        super(AlertDialog, self).__init__(**kwargs)
        self.message = message

    def dismiss_popup(self):
        self.dismiss()

# Define the Screens

            
class LoginScreen(Screen):
    def login(self, username, password):
        if username == '' and password == '':
            self.manager.current = 'main_menu'
            self.ids.login_status.text = ""
            self.ids.username_input.text = ""  # Clear the username field
            self.ids.password_input.text = ""  # Clear the password field
        else:
            self.ids.login_status.text = "Incorrect username or password."
        
class MainMenuScreen(Screen):
    pass

class AlertsScreen(Screen):
    def __init__(self, **kwargs):
        super(AlertsScreen, self).__init__(**kwargs)
        # Initialize any variables or states here if necessary

    def on_toggle_button_state_geo(self, state):
        global geo_state
        # Handle Geo Alert toggle state change
        geo_alert_status = self.ids['geo_alert_status']
        if state == 'down':
            geo_alert_status.text = 'Geo Alert: OFF'
            geo_state='down'
        else:
            geo_alert_status.text = 'Geo Alert: ON'
            geo_state='normal'

    def on_toggle_button_state_health(self, state):
        # Handle Health Alert toggle state change
        health_alert_status = self.ids['health_alert_status']
        if state == 'down':
            health_alert_status.text = 'Health Alert: OFF'
        else:
            health_alert_status.text = 'Health Alert: ON'

    def on_toggle_button_state_activity(self, state):
        # Handle Extreme Activity Alert toggle state change
        activity_alert_status = self.ids['activity_alert_status']
        if state == 'down':
            activity_alert_status.text = 'Extreme Activity Alert: OFF'
        else:
            activity_alert_status.text = 'Extreme Activity Alert: ON'


class HealthScreen(Screen):
    def on_enter(self, *args):
        super(HealthScreen, self).on_enter(*args)  # Call the superclass method to ensure proper behavior
        self.start_heartbeat_animation()
        threading.Thread(target=self.fetch_heart_data_continuously, args=()).start()
        
    
    def fetch_heart_data_continuously(self):
        while self.manager.current == 'health':
                # Simulate fetching new location data periodically (e.g., every few seconds)
                # Replace this with actual fetching logic as needed
            fetch_location_data(self.update_heart)
            global glo_heart_rate
            time.sleep(120/glo_heart_rate)  # Adjust the sleep time as needed for your application
    
    def update_heart(self, lat,lng,steps1,heart1):
        def ui_update3(dt):
                # Clear any existing marker from the map before adding a new one
            self.ids.heart_rate.text = f'Heart rate: {heart1}'
            global glo_heart_rate
            glo_heart_rate=int(heart1)
            print(f"{glo_heart_rate}")
            # Schedule the UI update to run in the main thread
            self.start_heartbeat_animation()
        from kivy.clock import Clock
        Clock.schedule_once(ui_update3)
        
    def start_heartbeat_animation(self):
        global glo_heart_rate
        heart = self.ids.heart  # Access the HeartBeatWidget by its id
        if glo_heart_rate==0:
            glo_heart_rate=1
        anim = Animation(size_x=150, size_y=150, d=60/glo_heart_rate,) + Animation(size_x=100, size_y=100, d=60/glo_heart_rate)
        anim.start(heart)
    



class LocationScreen(Screen):
    marker = None  # Initialize a class variable to keep track of the current marker
    cicrclemarker = None
    safe_zone_center = (39.872121, 32.761761)  # Predefined center for the safe zone
    safe_zone_radius = 0.01  # Default radius in degrees (adjust as needed)
    safe_zone_layer = None
    setting_safe_zone = False
    alert_shown = False
    
    def on_enter(self, *args):
        super(LocationScreen, self).on_enter(*args)
        # Ensure any existing marker is cleared when entering the screen
        if self.marker:
            self.ids.mapview.remove_widget(self.marker)
            self.marker = None
        # Start fetching data in a separate thread
        threading.Thread(target=self.fetch_location_data_continuously, args=()).start()
        self.safe_zone_center = 39.872121, 32.761761
        self.safe_zone_layer = None  # Initialize safe_zone_layer here
        

    def update_safe_zone_radius(self, radius_text):
        try:
            # Convert input text to float for the radius
            self.alert_shown = False
            self.safe_zone_radius = float(radius_text) / 111000  # Convert meters to degrees
            popup = Popup(title='Safe Zone Defined',
                          content=Label(text='Safe zone set with radius {} meters.'.format(int(self.safe_zone_radius * 111000))),
                          size_hint=(None, None), size=(400, 200))
            popup.open()
        except ValueError:
            popup = Popup(title='Safe Zone Cannot Be Defined',
                          content=Label(text='Safe zone value error of meters.'),
                          size_hint=(None, None), size=(400, 200))
            popup.open()
            
    
    def fetch_location_data_continuously(self):
        while self.manager.current == 'location':
            # Simulate fetching new location data periodically (e.g., every few seconds)
            # Replace this with actual fetching logic as needed
            fetch_location_data(self.update_map_with_location)
            time.sleep(5)  # Adjust the sleep time as needed for your application


    def update_map_with_location(self, lat,lng,steps1,heart1):
        def ui_update(dt):
            global geo_state
            # Clear any existing marker from the map before adding a new one
            if self.marker:
                self.ids.mapview.remove_widget(self.marker)
            # Create a new marker for the new location

            self.marker = MapMarkerPopup(lat=lat, lon=lng, source=marker_source)
            self.ids.mapview.add_widget(self.marker)
            
            # Center the map on the new marker
        
            
            self.ids.mapview.center_on(float(lat), float(lng))
            
            distance = self.calculate_distance(float(lat),float(lng), *self.safe_zone_center)
            print(f"Current location: ({lat}, {lng})")
            print(f"Safe zone center: {self.safe_zone_center}")
            print(f"Distance to safe zone: {distance * 111000:.2f} meters")
            if distance > self.safe_zone_radius and not self.alert_shown and geo_state=='normal':
                self.show_alert(f'Location is {distance * 111000:.2f} meters to the safe center.')
                self.alert_shown = True 
                update_cloud_data('com',1)
            if distance < self.safe_zone_radius:
                update_cloud_data('com',0)
        # Schedule the UI update to run in the main thread
        from kivy.clock import Clock
        Clock.schedule_once(ui_update)
                
    def calculate_distance(self, lat1, lon1, lat2, lon2):
        # Calculate distance between two points in degrees (Haversine formula)
        lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
        dlat = (lat2 - lat1)* math.pi/ 180
        dlon = (lon2 - lon1)* math.pi/ 180
        print(f"Distance lat lon: ({dlat}, {dlon})")
        a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
        c = 2 * math.atan2(math.sqrt(a),math.sqrt(1-a))
        r = 6371  # Radius of the Earth in kilometers
        return c * r/2 # Return distance in meters

    def show_alert(self, info):
        # Display an alert to the user
        popup = AlertDialog(message=info)
        popup.open()

    def dismiss_alert(self, instance):
        # Dismiss the alert popup
        instance.parent.parent.dismiss()
            

class AlertPopup(Popup):
    pass
class CustomProgressBar(ProgressBar):
    pass

class ActivityTrackingScreen(Screen):
    def compare_goal_with_steps(self):
        goal = App.get_running_app().root.get_screen('daily_goals').daily_goal
        steps_count = int(self.ids.step_count.text.split(': ')[1])  # Extract steps count from the label text
       
        progress_percentage = (steps_count / goal) * 100
        self.ids.custom_progress_bar.value = progress_percentage
        
        meters = int(steps_count*0.4)
        self.ids.distance_tracked.text = f'SteDistance: {meters}'
        if progress_percentage >= 100:
            popup = Popup(title='Daily Goal!',
                          content=Label(text='Congratulations!\nYou ve reached your daily goal!'),
                          size_hint=(None, None), size=(400, 200))
            popup.open()
        if progress_percentage < 100:
            popup = Popup(title='Daily Goal!',
                          content=Label(text='Keep going!\nYou havent reached your daily goal yet.'),
                          size_hint=(None, None), size=(400, 200))
            popup.open()

    def on_enter(self, *args):
        
        super(ActivityTrackingScreen, self).on_enter(*args)
        # Ensure any existing marker is cleared when entering the screen
        
        threading.Thread(target=self.fetch_steps_data_continuously, args=()).start()
        
    
    def fetch_steps_data_continuously(self):
        while self.manager.current == 'activity_tracking':
                # Simulate fetching new location data periodically (e.g., every few seconds)
                # Replace this with actual fetching logic as needed
            fetch_location_data(self.update_steps)
            time.sleep(5)  # Adjust the sleep time as needed for your application
    
    
    def update_steps(self, lat,lng,steps1,heart1):
        def ui_update2(dt):
                # Clear any existing marker from the map before adding a new one
            self.ids.step_count.text = f'Steps: {steps1}'
            # Schedule the UI update to run in the main thread
        from kivy.clock import Clock
        Clock.schedule_once(ui_update2)
        
class SettingsScreen(Screen):
    pass
class GoalsScreen(Screen):
    daily_goal = 1000
    def update_daily_goal(self, goal_text):
        try:
            self.alert_shown = False
            self.daily_goal = int(goal_text) 
            popup = Popup(title='Daily Goal Set!',
                          content=Label(text='Daily goal set with {} steps!'.format(int(self.daily_goal))),
                          size_hint=(None, None), size=(400, 200))
            popup.open()
        except ValueError:
            popup = Popup(title='Daily Goal Cannot Be Defined',
                          content=Label(text='Daily goal value error of steps.'),
                          size_hint=(None, None), size=(400, 200))
            popup.open()


class PetInfoScreen(Screen):
    def save_pet_info(self, pet_type, breed, age, gender):
        # Save pet information logic here
        print(f"Saving pet info: Type={pet_type}, Breed={breed}, Age={age}, Gender={gender}")
        try:
            popup = Popup(title='Success', content=Label(text='Pet Information saved successfully!'),
                            size_hint=(None, None), size=(400, 200))
            popup.open()
        except:
            popup = Popup(title='Error', content=Label(text='Failed to save Pet Information!'),
                        size_hint=(None, None), size=(400, 200))
            popup.open()

    def on_pet_type_select(self, pet_type):
        dog_breeds = ['Labrador Retriever', 'German Shepherd', 'Golden Retriever']
        cat_breeds = ['Persian Cat', 'Siamese Cat', 'Maine Coon']
        
        if pet_type == 'Dog':
            self.ids.pet_breed.values = dog_breeds
        elif pet_type == 'Cat':
            self.ids.pet_breed.values = cat_breeds
        else:
            self.ids.pet_breed.values = []
        
        self.ids.pet_breed.text = 'Select Breed'

class WiFiSettingsScreen(Screen):
    def save_wifi_settings(self, ssid, password):
        try:
            # Assuming 'Settings' is a collection and 'wifi_settings' is a document where WiFi settings are stored
            doc_ref = db.collection('your_collection').document('your_document')
            # Set the ssid and password in the document
            doc_ref.set({
                'ssid': ssid,
                'password': password
            }, merge=True)  # Use merge=True to update the document without overwriting other fields
            # Inform the user of success
            popup = Popup(title='Success', content=Label(text='WiFi settings saved successfully!'),
                        size_hint=(None, None), size=(400, 200))
            popup.open()
        except Exception as e:
            # Log the error and inform the user
            print(f'An error occurred: {e}')
            popup = Popup(title='Error', content=Label(text='Failed to save WiFi settings.'),
                        size_hint=(None, None), size=(400, 200))
            popup.open()

# Create the screen manager
sm = ScreenManager()
sm.add_widget(LoginScreen(name='login'))
sm.add_widget(MainMenuScreen(name='main_menu'))
sm.add_widget(LocationScreen(name='location'))
sm.add_widget(HealthScreen(name='health'))
sm.add_widget(ActivityTrackingScreen(name='activity_tracking'))
sm.add_widget(SettingsScreen(name='Settings'))
sm.add_widget(PetInfoScreen(name='pet_info'))
sm.add_widget(GoalsScreen(name='daily_goals'))
sm.add_widget(AlertsScreen(name='alerts'))
sm.add_widget(WiFiSettingsScreen(name='wifi_settings'))



class MyApp(App):

    def build(self):
        return Builder.load_string(kv)
    

if __name__ == '__main__':
    MyApp().run()

[DEBUG  ] STREAM b'IHDR' 16 13
[DEBUG  ] STREAM b'sRGB' 41 1
[DEBUG  ] STREAM b'bKGD' 54 6
[DEBUG  ] b'bKGD' 54 6 (unknown)
[DEBUG  ] STREAM b'pHYs' 72 9
[DEBUG  ] STREAM b'tIME' 93 7
[DEBUG  ] b'tIME' 93 7 (unknown)
[DEBUG  ] STREAM b'IDAT' 112 8192
[INFO   ] [GL          ] NPOT texture support is available
[INFO   ] [Base        ] Start application main loop
[DEBUG  ] [Making request] POST https://oauth2.googleapis.com/token
[DEBUG  ] [Starting new HTTPS connection (1)] oauth2.googleapis.com:443
[DEBUG  ] [https          ]//oauth2.googleapis.com:443 "POST /token HTTP/1.1" 200 None


Saving pet info: Type=Dog, Breed=Labrador Retriever, Age=5, Gender=Male
